✅ 4. ParentDocumentRetriever

Use Case: Retrieve parent chunks based on a retrieval of smaller child chunks, great for keeping context intact in long documents.

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from dotenv import load_dotenv
import os

# 1. Load environment
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# 1. Prepare documents
docs = [Document(page_content="LangChain helps build LLM-powered apps with memory and agents.", metadata={"id": "1"}),
        Document(page_content="Agents in LangChain use tools to answer questions.", metadata={"id": "2"})]

# 2. Setup child splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

# 3. Setup vectorstore for children
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)

# 4. Parent retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=InMemoryStore(),  # Stores parent docs
    child_splitter=child_splitter
)

# 5. Add documents
retriever.add_documents(docs)

# 6. Retrieve
results = retriever.get_relevant_documents("What are agents?")
for doc in results:
    print("📄 Retrieved Doc:", doc.page_content)


C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_10804\1140793398.py:21: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_10804\1140793398.py:35: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents("What are agents?")


📄 Retrieved Doc: Agents in LangChain use tools to answer questions.
📄 Retrieved Doc: LangChain helps build LLM-powered apps with memory and agents.


✅ 5. BM25Retriever

Use Case: Pure keyword-based search (like traditional search engines). No embeddings needed.

In [5]:
#!pip install rank_bm25

In [2]:
from langchain.retrievers import BM25Retriever
from langchain.schema.document import Document

# Create simple text docs
docs = [
    Document(page_content="LangChain enables LLM applications."),
    Document(page_content="Vector search is powerful."),
    Document(page_content="BM25 is a classical retrieval method.")
]

# Create BM25 retriever
bm25_retriever = BM25Retriever.from_documents(docs)

# Retrieve
results = bm25_retriever.get_relevant_documents("How does BM25 work?")
for doc in results:
    print("📝 BM25 Result:", doc.page_content)


📝 BM25 Result: BM25 is a classical retrieval method.
📝 BM25 Result: Vector search is powerful.
📝 BM25 Result: LangChain enables LLM applications.


✅ 6. EnsembleRetriever

Use Case: Combine multiple retrievers (e.g., keyword + vector-based) with weighted scores.

In [3]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.document import Document

# Sample docs
docs = [Document(page_content="LangChain supports LLMs."), Document(page_content="You can build AI apps using LangChain.")]

# BM25 Retriever
bm25 = BM25Retriever.from_documents(docs)

# Vector Retriever
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)
vector_retriever = vectorstore.as_retriever()

# Ensemble Retriever (equal weight)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, vector_retriever],
    weights=[0.5, 0.5]
)

# Query
results = ensemble_retriever.get_relevant_documents("AI apps using LangChain")
for doc in results:
    print("🔍 Ensemble Doc:", doc.page_content)


🔍 Ensemble Doc: You can build AI apps using LangChain.
🔍 Ensemble Doc: LangChain supports LLMs.


7.TimeWeightedVectorStoreRetriever

#This retriever boosts document relevance by factoring recency and importance of interactions (used in agents with memory or relevance ranking).

In [1]:
from dotenv import load_dotenv
import os
from datetime import datetime

# Load environment variables
load_dotenv()

from langchain_openai import OpenAIEmbeddings   # <-- FIXED IMPORT
from langchain.vectorstores import FAISS
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import Document

# Initialize embeddings
embedding = OpenAIEmbeddings()

# Sample documents
docs = [
    Document(page_content="LangChain is for LLM-based apps", metadata={"last_accessed_at": datetime.now()}),
    Document(page_content="Vector search improves relevance", metadata={"last_accessed_at": datetime.now()}),
]

# Create vectorstore
vectorstore = FAISS.from_documents(docs, embedding)

# Create retriever
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore,
    decay_rate=0.01,
    k=5,
    search_kwargs={"k": 5}   # <-- ensures results return
)

# Retrieve
results = retriever.get_relevant_documents("LangChain")

print("\nRetrieved Documents:\n")
for r in results:
    print("---- Document ----")
    print("Content:", r.page_content)
    print("Metadata:", r.metadata)
    print("")


C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_17420\1472465846.py:34: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents("LangChain")



Retrieved Documents:



In [24]:
#!pip install tavily-python

8.TavilySearchAPIRetriever

In [5]:
from langchain.retrievers import TavilySearchAPIRetriever
import os

# Set your Tavily API key
#os.environ["TAVILY_API_KEY"] = "your_tavily_api_key"

from dotenv import load_dotenv
import os
# 🔐 Load API keys
load_dotenv(".env")
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# Initialize Tavily Retriever
retriever = TavilySearchAPIRetriever(k=3)

# Perform search
docs = retriever.get_relevant_documents("Latest updates about LangChain")
for doc in docs:
    print(doc.page_content)


... LangChain 1.1. LangChain v1.1.0 is now available This release makes agent development more reliable, more structured, and more context-aware — powered by a new
The langchain namespace has been streamlined to focus on essential building blocks for agents, with legacy functionality moved to langchain-classic . To upgrade
LangChain provides the engineering platform and open source frameworks developers use to build, test, and deploy reliable AI agents.
